<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Getting Started With Spark using Python**


Estimated time needed: **15** minutes


![](http://spark.apache.org/images/spark-logo.png)


### The Python API


Spark is written in Scala, which compiles to Java bytecode, but you can write python code to communicate to the java virtual machine through a library called py4j. Python has the richest API, but it can be somewhat limiting if you need to use a method that is not available, or if you need to write a specialized piece of code. The latency associated with communicating back and forth to the JVM can sometimes cause the code to run slower.
An exception to this is the SparkSQL library, which has an execution planning engine that precompiles the queries. Even with this optimization, there are cases where the code may run slower than the native scala version.
The general recommendation for PySpark code is to use the "out of the box" methods available as much as possible and avoid overly frequent (iterative) calls to Spark methods. If you need to write high-performance or specialized code, try doing it in scala.
But hey, we know Python rules, and the plotting libraries are way better. So, it's up to you!


## Objectives


In this lab, we will go over the basics of Apache Spark and PySpark. We will start with creating the SparkContext and SparkSession. We then create an RDD and apply some basic transformations and actions. Finally we demonstrate the basics dataframes and SparkSQL.

After this lab you will be able to:

*   Create the SparkContext and SparkSession
*   Create an RDD and apply some basic transformations and actions to RDDs
*   Demonstrate the use of the basics Dataframes and SparkSQL


***


## Setup


For this lab, we are going to be using Python and Spark (PySpark). These libraries should be installed in your lab environment or in SN Labs.


In [2]:
# Installing required packages
!pip install pyspark
!pip install findspark

     |████████████████████████████████| 281.3 MB 54 kB/s s eta 0:00:01
     |████████████████████████████████| 198 kB 51.7 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805914 sha256=22bcccba3f75565d4ae217f19912320c8e26f37d8c98229395c3ca586e0aabda
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/23/f6/d3/110e53bd43baeb8d7d38049733d48e39cbecd056f01dba7ee8
Successfully built pyspark


In [4]:
import findspark
findspark.init()

ValueError: Couldn't find Spark, make sure SPARK_HOME env is set or Spark is in an expected location (e.g. from homebrew installation).

In [5]:
# PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the spark context. 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

## Exercise 1 -  Spark Context and Spark Session


In this exercise, you will create the Spark Context and initialize the Spark session needed for SparkSQL and DataFrames.
SparkContext is the entry point for Spark applications and contains functions to create RDDs such as `parallelize()`. SparkSession is needed for SparkSQL and DataFrame operations.


#### Task 1: Creating the spark session and context


In [6]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/20 21:48:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Task 2: Initialize Spark session

To work with dataframes we just need to verify that the spark session instance has been created.


In [7]:
spark

## Exercise 2: RDDs

In this exercise we work with Resilient Distributed Datasets (RDDs). RDDs are Spark's primitive data abstraction and we use concepts from functional programming to create and manipulate RDDs.


#### Task 1: Create an RDD.

For demonstration purposes, we create an RDD here by calling `sc.parallelize()`\
We create an RDD which has integers from 1 to 30.


In [8]:
data = range(1,30)
# print first element of iterator
print(data[0])
len(data)
xrangeRDD = sc.parallelize(data, 4)

# this will let us know that we created an RDD
xrangeRDD

1


PythonRDD[1] at RDD at PythonRDD.scala:53

#### Task 2: Transformations


A transformation is an operation on an RDD that results in a new RDD. The transformed RDD is generated rapidly because the new RDD is lazily evaluated, which means that the calculation is not carried out when the new RDD is generated. The RDD will contain a series of transformations, or computation instructions, that will only be carried out when an action is called. In this transformation, we reduce each element in the RDD by 1. Note the use of the lambda function. We also then filter the RDD to only contain elements <10.


In [9]:
subRDD = xrangeRDD.map(lambda x: x-1)
filteredRDD = subRDD.filter(lambda x : x<10)


#### Task 3: Actions


A transformation returns a result to the driver. We now apply the `collect()` action to get the output from the transformation.


In [10]:
print(filteredRDD.collect())
filteredRDD.count()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


10

#### Task 4: Caching Data


This simple example shows how to create an RDD and cache it. Notice the **10x speed improvement**!  If you wish to see the actual computation time, browse to the Spark UI...it's at host:4040.  You'll see that the second calculation took much less time!


In [11]:
import time 

test = sc.parallelize(range(1,50000),4)
test.cache()

t1 = time.time()
# first count will trigger evaluation of count *and* cache
count1 = test.count()
dt1 = time.time() - t1
print("dt1: ", dt1)


t2 = time.time()
# second count operates on cached data only
count2 = test.count()
dt2 = time.time() - t2
print("dt2: ", dt2)


test.count()

dt1:  0.6106066703796387
dt2:  0.1908738613128662


49999

## Exercise 3: DataFrames and SparkSQL


In order to work with the extremely powerful SQL engine in Apache Spark, you will need a Spark Session. We have created that in the first Exercise, let us verify that spark session is still active.


In [12]:
spark

#### Task 1: Create Your First DataFrame!


You can create a structured data set (much like a database table) in Spark.  Once you have done that, you can then use powerful SQL tools to query and join your dataframes.


In [13]:
# Download the data first into a local `people.json` file
!curl https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/people.json >> people.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    73  100    73    0     0    588      0 --:--:-- --:--:-- --:--:--   588


In [14]:
# Read the dataset into a spark dataframe using the `read.json()` function
df = spark.read.json("people.json").cache()

In [15]:
# Print the dataframe as well as the data schema
df.show()
df.printSchema()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [16]:
# Register the DataFrame as a SQL temporary view
df.createTempView("people")

#### Task 2: Explore the data using DataFrame functions and SparkSQL

In this section, we explore the datasets using functions both from dataframes as well as corresponding SQL queries using sparksql. Note the different ways to achieve the same task!


In [17]:
# Select and show basic data columns

df.select("name").show()
df.select(df["name"]).show()
spark.sql("SELECT name FROM people").show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



In [18]:
# Perform basic filtering

df.filter(df["age"] > 21).show()
spark.sql("SELECT age, name FROM people WHERE age > 21").show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [19]:
# Perfom basic aggregation of data

df.groupBy("age").count().show()
spark.sql("SELECT age, COUNT(age) as count FROM people GROUP BY age").show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|  30|    1|
|null|    1|
+----+-----+

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|  30|    1|
|null|    0|
+----+-----+



***


### Question 1 - RDDs


Create an RDD with integers from 1-50. Apply a transformation to multiply every number by 2, resulting in an RDD that contains the first 50 even numbers.


In [ ]:
# starter code
# numbers = range(1, 50)
# numbers_RDD = ...
# even_numbers_RDD = numbers_RDD.map(lambda x: ..)

In [20]:
# Code block for learners to answer

data = range(1, 51)
print(data)
numbers_RDD = sc.parallelize(data,4)

print(numbers_RDD.collect())

even_numbers_RDD = numbers_RDD.map(lambda x: x * 2 )

filteredRDD = even_numbers_RDD.filter(lambda x:x<=100)

print(filteredRDD.collect())

range(1, 51)
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100]


### Question 2 - DataFrames and SparkSQL


Similar to the `people.json` file, now read the `people2.json` file into the notebook, load it into a dataframe and apply SQL operations to determine the average age in our people2 file.


In [26]:
# starter code
# !curl https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/people2.json >> people2.json
# df = spark.read...
# df.createTempView..
# spark.sql("SELECT ...")

In [52]:
# Code block for learners to answer
!curl https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/people.json >> people2.json
df = spark.read.json("people2.json")
df.createTempView("people2")
df.select("age").show()
spark.sql("SELECT avg(age) from people2 WHERE age > 0").show()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    73  100    73    0     0    722      0 --:--:-- --:--:-- --:--:--   722
+----+
| age|
+----+
|null|
|  30|
|  19|
+----+

+--------+
|avg(age)|
+--------+
|    24.5|
+--------+



Double-click **here** for a hint.

<!-- The hint is below:

1. The SQL query "Select AVG(column_name) from.." can be used to find the average value of a column. 
2. Another possible way is to use the dataframe operations select() and mean()
-->


Double-click **here** for the solution.

<!-- The answer is below:
df = spark.read('people2.json')
df.createTempView("people2")
spark.sql("SELECT AVG(age) from people2")
-->


### Question 3 - SparkSession


Close the SparkSession we created for this notebook


In [53]:
# Code block for learners to answer
spark.stop()

In [54]:
spark

AttributeError: 'NoneType' object has no attribute 'sc'

Double-click **here** for the solution.

<!-- The answer is below:

spark.stop() will stop the spark session

-->


## Authors


[Karthik Muthuraman](https://www.linkedin.com/in/karthik-muthuraman/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMBD0225ENSkillsNetwork25716109-2021-01-01)


### Other Contributors


[Jerome Nilmeier](https://github.com/nilmeier)


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description |
| ----------------- | ------- | ---------- | ------------------ |
| 2021-07-02        | 0.2     | Karthik    | Beta launch        |
| 2021-06-30        | 0.1     | Karthik    | First Draft        |


Copyright © 2021 IBM Corporation. All rights reserved.
